In [21]:
import simulation as sim
from importlib import reload
import numpy as np
import pickle
import random
from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd
import os

In [6]:
trials = pd.read_pickle('trials_5.pkl')
runs = pd.DataFrame(trials.vals)
runs['loss'] = trials.losses()
runs.sort_values('loss', inplace=True)

In [19]:
runs[runs.loss < -0.8][['A_eq_star_scaling', 'P_star', 'alpha', 'beta']].describe().loc[['min', 'max']]

,A_eq_star_scaling,P_star,alpha,beta
min,0.078842,460.438517,0.000147,0.008351
max,0.387942,904.892311,0.004797,0.999717


In [23]:
def instantiate_and_run_simulation(
    run_number
):
    simulation = sim.Simulation(N_bodies=10, timestep_reset=1000)

    beta = random.uniform(0.008351, 0.999717)
    alpha =  random.uniform(0.000147, 0.004797)
    A_eq_star_scaling = random.uniform(0.078842, 0.387942)
    P_star = random.uniform(460.438517, 904.892311) 
    radius_scaling = random.uniform(0.5, 1.2)

    simulation.execute(
        beta=beta,
        alpha=alpha,
        A_eq_star_scaling=A_eq_star_scaling,
        P_star=P_star,
        radius_scaling=radius_scaling,
        write_results=True,
        max_reset_count=20,
        run_number=run_number,
        write_path="f:\\Bel_Simulation\\outputs_10")

In [24]:
with sim.tqdm_joblib(tqdm(desc="My Simulation", total=1_000_000)) as progress_bar:
    Parallel(n_jobs=20)(delayed(instantiate_and_run_simulation)(j) for j in range(1_000_000))

My Simulation:   0%|          | 3344/1000000 [72:50:17<21708:55:39, 78.41s/it] 


MemoryError: 

In [ ]:
results = pd.concat([pd.read_parquet(f"C:\\Users\\Tom\\Documents\\Bel PhD\\Bel_Simulation\\outputs\\{i}") for i in os.listdir("C:\\Users\\Tom\\Documents\\Bel PhD\\Bel_Simulation\\outputs") if i.endswith(".parquet")])